In [6]:
import pandas as pd

df = pd.read_csv('../data/班分け結果_R2_名前付き_50音順.csv')
df_key = pd.read_csv('../data/名簿一覧_Bridge_ID割当済_最終版.csv')

In [7]:
# dfのメンバー名をキーにして、df_keyから所属会社(内定先会社)とBridge IDを取得して結合する
required_left = {'メンバー名'}
required_right = {'お名前(漢字)', '所属会社(内定先会社)', 'Bridge ID'}
if not required_left.issubset(df.columns):
    raise KeyError(f"dfに必要なカラムがありません: {required_left - set(df.columns)}")
if not required_right.issubset(df_key.columns):
    raise KeyError(f"df_keyに必要なカラムがありません: {required_right - set(df_key.columns)}")

df = df.merge(
    df_key[['お名前(漢字)', '所属会社(内定先会社)', 'Bridge ID']],
    left_on='メンバー名',
    right_on='お名前(漢字)',
    how='left'
)

# 重複したお名前カラムを削除（必要なら残す）
df.drop(columns=['お名前(漢字)'], inplace=True)

# Bridge IDを4桁のゼロ埋め文字列にする（表示/出力で4桁にしたい場合）
df['Bridge ID'] = df['Bridge ID'].astype(int).astype(str).str.zfill(4)

# Bridge IDで昇順ソート（数値としてソートしてから元のBridge ID表記は保持）
df['__BridgeID_int'] = pd.to_numeric(df['Bridge ID'], errors='coerce').fillna(0).astype(int)
df = df.sort_values(by='__BridgeID_int', ascending=True).reset_index(drop=True)
df.drop(columns='__BridgeID_int', inplace=True)

# Bridge IDで重複を削除して1件にする（先頭を保持）
_before = len(df)
df = df.drop_duplicates(subset='Bridge ID', keep='first').reset_index(drop=True)
_after = len(df)
print(f"Removed {_before - _after} duplicate rows by Bridge ID. Remaining rows: {_after}")

# 確認
df.head()

df.to_csv('../data/班分け結果_R2_ID付き_ID順.csv', index=False)

Removed 2 duplicate rows by Bridge ID. Remaining rows: 192
